In [4]:
import os
import cv2
import numpy as np

# 定义参数
num_rows, num_cols = 16, 16
image_height, image_width = 240, 320  # 每张图像的分辨率
input_folder = './toyLF'  # 输入文件夹
output_image_path = './light_field_matrix.png'  # 输出矩阵图像的路径

# 初始化用于存储图像的 NumPy 数组
light_field_images = np.zeros((num_rows, num_cols, image_height, image_width, 3), dtype=np.uint8)

def load_images_to_matrix(folder):
    counter = 1
    for row in range(num_rows):
        for col in range(num_cols):
            filename = os.path.join(folder, f'lowtoys{counter:03d}.bmp')
            img = cv2.imread(filename)
            if img is not None:
                img = cv2.resize(img, (image_width, image_height))  # 确保图像大小一致
                light_field_images[row, col] = img  # 将图像存储在数组中
                counter += 1
            else:
                print(f"图像 {filename} 未找到")

def create_matrix_image():
    # 创建一个大的图像矩阵
    matrix_image = np.zeros((num_rows * image_height, num_cols * image_width, 3), dtype=np.uint8)
    for row in range(num_rows):
        for col in range(num_cols):
            matrix_image[row * image_height:(row + 1) * image_height,
                         col * image_width:(col + 1) * image_width] = light_field_images[row, col]
    return matrix_image

load_images_to_matrix(input_folder)
matrix_image = create_matrix_image()

# 保存矩阵图像为 PNG 格式
cv2.imwrite(output_image_path, matrix_image)

print("图像矩阵生成完成并已保存为 PNG！")


图像矩阵生成完成并已保存为 PNG！
